In [1]:
from pathlib import Path
import sys

# Make this path be the project's "base" directory, so we can include modules
notebook_directory_ancestor = Path.cwd().resolve().parent.parent.parent.parent
print(notebook_directory_ancestor)
core_code_directory = notebook_directory_ancestor / "PythonLibraries" / "CoreCode"

is_core_code_directory_in_sys_path = str(core_code_directory) in sys.path
is_notebook_directory_ancestor_in_sys_path = str(notebook_directory_ancestor) in sys.path
print("Is CoreCode directory in sys.path?", is_core_code_directory_in_sys_path)
print("Is notebook directory's ancestor in sys.path?", is_notebook_directory_ancestor_in_sys_path)

if not is_core_code_directory_in_sys_path:
    sys.path.append(str(core_code_directory))

/InServiceOfX
Is CoreCode directory in sys.path? False
Is notebook directory's ancestor in sys.path? False


In [2]:
from corecode.FileIO import (GetJSONAsDict,)
from corecode.Utilities import (
    DataSubdirectories,
    LoadConfigurationFile)

In [3]:
data_sub_dirs = DataSubdirectories()

In [4]:
from insightface.app import FaceAnalysis

/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [7]:
# prepare 'antelopev2'
app = FaceAnalysis(
    name='antelopev2',
    root=str(data_sub_dirs.ModelsDiffusion / "InstantX"),
    #providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
    providers=['CUDAExecutionProvider',])
app.prepare(ctx_id=0, det_size=(384, 384))

download_path: /Data/Models/Diffusion/InstantX/models/antelopev2


RuntimeError: Failed downloading url https://github.com/deepinsight/insightface/releases/download/v0.7/antelopev2.zip

In [12]:
%time
from diffusers.models import ControlNetModel
import torch

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.39 µs


In [10]:
controlnet_path=data_sub_dirs.ModelsDiffusion / "InstantX" / "InstantID" / "ControlNetModel"
print(controlnet_path)

/Data/Models/Diffusion/InstantX/InstantID/ControlNetModel


In [13]:
controlnet = ControlNetModel.from_pretrained(str(controlnet_path), torch_dtype=torch.float16)

In [15]:
print(sys.path)

['/InServiceOfX/Notebooks/ThirdParty/HuggingFace/diffusers', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/ThirdParty/accelerate/src', '/ThirdParty/datasets/src', '/ThirdParty/diffusers/src', '/ThirdParty/transformers/src', '/usr/local/lib/python3.10/dist-packages/nvfuser-0.1.1+gitunknown-py3.10-linux-x86_64.egg', '/usr/lib/python3/dist-packages', '/InServiceOfX/PythonLibraries/CoreCode', '/tmp/tmpednqsq5c']


In [17]:
instant_id_path = notebook_directory_ancestor.parent / "ThirdParty" / "InstantID"
if (instant_id_path.exists() and str(instant_id_path) not in sys.path):
    sys.path.append(str(instant_id_path))

In [18]:
from pipeline_stable_diffusion_xl_instantid import StableDiffusionXLInstantIDPipeline, draw_kps

In [21]:
model_subdir = data_sub_dirs.ModelsDiffusion / "stabilityai" / "stable-diffusion-xl-base-1.0"
print(model_subdir)
print(model_subdir.exists())

/Data/Models/Diffusion/stabilityai/stable-diffusion-xl-base-1.0
True


In [22]:
%time
pipe = StableDiffusionXLInstantIDPipeline.from_pretrained(
    str(model_subdir),
    controlnet=controlnet,
    torch_dtype=torch.float16,
    local_files_only=True,)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

cached_folder:  /Data/Models/Diffusion/stabilityai/stable-diffusion-xl-base-1.0
name:  text_encoder_2
{'torch_dtype': torch.float16, 'device_map': None, 'max_memory': None, 'offload_folder': None, 'offload_state_dict': False, 'low_cpu_mem_usage': True}
cached_folder:  /Data/Models/Diffusion/stabilityai/stable-diffusion-xl-base-1.0
name:  scheduler
{}
cached_folder:  /Data/Models/Diffusion/stabilityai/stable-diffusion-xl-base-1.0
name:  vae
{'torch_dtype': torch.float16, 'device_map': None, 'max_memory': None, 'offload_folder': None, 'offload_state_dict': False, 'variant': None, 'low_cpu_mem_usage': True}
cached_folder:  /Data/Models/Diffusion/stabilityai/stable-diffusion-xl-base-1.0
name:  tokenizer
{}
cached_folder:  /Data/Models/Diffusion/stabilityai/stable-diffusion-xl-base-1.0
name:  text_encoder
{'torch_dtype': torch.float16, 'device_map': None, 'max_memory': None, 'offload_folder': None, 'offload_state_dict': False, 'low_cpu_mem_usage': True}
cached_folder:  /Data/Models/Diffusio

In [24]:
%time
pipe.cuda()

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 8.82 µs


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 5.93 GiB of which 8.62 MiB is free. Process 149401 has 4.42 GiB memory in use. Of the allocated memory 4.20 GiB is allocated by PyTorch, and 130.72 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [6]:
(data_sub_dirs.ModelsDiffusion / "InstantX").exists()

True

In [8]:
help(FaceAnalysis)

Help on class FaceAnalysis in module insightface.app.face_analysis:

class FaceAnalysis(builtins.object)
 |  FaceAnalysis(name='buffalo_l', root='~/.insightface', allowed_modules=None, **kwargs)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name='buffalo_l', root='~/.insightface', allowed_modules=None, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  draw_on(self, img, faces)
 |  
 |  get(self, img, max_num=0)
 |  
 |  prepare(self, ctx_id, det_thresh=0.5, det_size=(640, 640))
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)

